<a href="https://colab.research.google.com/github/Bhavin6969/Traffic_sign_recognition/blob/main/Traffic_sign_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
import tensorflow as tf
from PIL import Image
import os
os.chdir("C:/Users/Bhavin Joshi/Desktop/New Folder")
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

In [ ]:
data=[]
labels=[]
classes=43
cur_path=os.getcwd()

# Preprocess the images

In [ ]:
for i in range(classes):
    path=os.path.join(cur_path,'train',str(i))
    images=os.listdir(path)
    for a in images:
        try:
            image=Image.open(path+'\\'+a)
            image=image.resize((30,30))
            image=np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

# Converting lists into numpy arrays 

In [ ]:
data=np.array(data)
labels=np.array(labels)

In [ ]:
os.mkdir('training')
np.save('./training/data',data)
np.save('./training/target',labels)

Load data and Labels

In [ ]:
data=np.load('./training/data.npy')
labels=np.load('./training/target.npy')

In [ ]:
print(data.shape,labels.shape)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data,labels,test_size=0.2,random_state=0)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
y_train=to_categorical(y_train,43)
y_test=to_categorical(y_test,43)

In [ ]:
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),activation='relu',input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32,kernel_size=(5,5),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
epochs=20
history=model.fit(X_train,y_train,batch_size=32,epochs=epochs,validation_data=(X_test,y_test))

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'],label='training accuracy')
plt.plot(history.history['val_accuracy'],label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

TESTING

In [ ]:
def testing(testcsv):
    y_test=pd.read_csv(testcsv)
    label=y_test['ClassId'].values
    imgs=y_test['Path'].values
    data=[]
    for img in imgs:
        image=Image.open(img)
        image=image.resize((30,30))
        data.append(np.array(image))
    x_test=np.array(data)
    return x_test,label

In [ ]:
X_test,label=testing('Test.csv')

In [ ]:
Y_pred = np.argmax(model.predict(X_test),axis=1)
Y_pred

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(label,Y_pred))

In [ ]:
model.save("./training/TSR.h5")

In [ ]:
import os
os.chdir("C:/Users/Bhavin Joshi/Desktop/New Folder")
from keras.models import load_model
model=load_model("./training/TSR.h5")

In [ ]:
classes = { 1:'Speed limit (20km/h)',
            2:'Speed limit (30km/h)', 
            3:'Speed limit (50km/h)', 
            4:'Speed limit (60km/h)', 
            5:'Speed limit (70km/h)', 
            6:'Speed limit (80km/h)', 
            7:'End of speed limit (80km/h)', 
            8:'Speed limit (100km/h)', 
            9:'Speed limit (120km/h)', 
            10:'No passing', 
            11:'No passing veh over 3.5 tons', 
            12:'Right-of-way at intersection', 
            13:'Priority road', 
            14:'Yield', 
            15:'Stop', 
            16:'No vehicles', 
            17:'Veh > 3.5 tons prohibited', 
            18:'No entry', 
            19:'General caution', 
            20:'Dangerous curve left', 
            21:'Dangerous curve right', 
            22:'Double curve', 
            23:'Bumpy road', 
            24:'Slippery road', 
            25:'Road narrows on the right', 
            26:'Road work', 
            27:'Traffic signals', 
            28:'Pedestrians', 
            29:'Children crossing', 
            30:'Bicycles crossing', 
            31:'Beware of ice/snow',
            32:'Wild animals crossing', 
            33:'End speed + passing limits', 
            34:'Turn right ahead', 
            35:'Turn left ahead', 
            36:'Ahead only', 
            37:'Go straight or right', 
            38:'Go straight or left', 
            39:'Keep right', 
            40:'Keep left', 
            41:'Roundabout mandatory', 
            42:'End of no passing', 
            43:'End no passing veh > 3.5 tons' }

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def test_on_img(img):
    data=[]
    image=Image.open(img)
    image=image.resize((30,30))
    data.append(np.array(image))
    X_test=np.array(data)
    Y_pred = np.argmax(model.predict(X_test),axis=1)
    return image,Y_pred

In [ ]:
plot,prediction=test_on_img("C:/Users/Bhavin Joshi/Desktop/New folder/Train/34/00034_00000_00002.png")
s=[str(i)for i in prediction]
a=int("".join(s))
print("Predicted Sign is",classes[a+1])
plt.imshow(plot)
plt.show()